In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from bs4 import BeautifulSoup
import time
import re
import pandas as pd

driver = webdriver.Chrome()
driver.get('https://www.cars24.com/buy-used-car?sort=bestmatch&serveWarrantyCount=true&gaId=2055791439.1722708746&storeCityId=777')

scroll_count = 3
for _ in range(scroll_count):
   driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
   time.sleep(12)

page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')
   
all_data = soup.find_all('a', class_='IIJDn')
details=[]
for car in all_data:
    try:
        car_name = car.find('h3', class_='_11dVb').get_text()
        match = re.match(r'(\d{4})\s+(.*)', car_name)
        if match:
            year = match.group(1)
            name = match.group(2)
        else:
            continue

        company = name.split(' ')[0]
        model = ' '.join(name.split(' ')[1:])
        location = 'Kolkata' #Change it to any city of your choice
        
        price = car.find('strong', class_='_3RL-I').text

        ul_element = car.find('ul', class_='_3J2G-')

        fuel_type = None
        km_driven = None
        transmission = None

        if ul_element:
            li_elements = ul_element.find_all('li')

            combined_text = ' '.join(li.get_text() for li in li_elements)

            km_match = re.search(r'([\d,]+)\s*km', combined_text)
            fuel_match = re.search(r'\b(Petrol|Diesel|CNG|Electric)\b', combined_text, re.IGNORECASE)
            transmission_match = re.search(r'\b(Manual|Automatic)\b', combined_text, re.IGNORECASE)

            if km_match:
                km_driven = km_match.group(1)
            if fuel_match:
                fuel_type = fuel_match.group(0)
            if transmission_match:
                transmission = transmission_match.group(0)

        details.append([name, model, company, year, fuel_type, km_driven, transmission, price, location])
    except Exception as e:
        print(f"Error processing car: {e}")

details = pd.DataFrame(details, columns=['Name', 'Model','Company','Year', 'Fuel Type', 'KM Driven', 'Transmission Type','Price', 'Location'])
details.to_csv('Cars_Kolkata.csv', index=False)
details.tail(20)

,Name,Model,Company,Year,Fuel Type,KM Driven,Transmission Type,Price,Location
99,Tata Tiago XZA PLUS PETROL,Tiago XZA PLUS PETROL,Tata,2023,Petrol,"7,889",Automatic,₹6.72 Lakh,Kolkata
100,Maruti Swift VXI,Swift VXI,Maruti,2019,Petrol,"23,964",Manual,₹4.85 Lakh,Kolkata
101,Maruti Alto LXI O,Alto LXI O,Maruti,2022,Petrol,"1,011",Manual,₹3.85 Lakh,Kolkata
102,Maruti Wagon R 1.0 VXI,Wagon R 1.0 VXI,Maruti,2018,Petrol,"38,242",Manual,₹3.63 Lakh,Kolkata
103,Hyundai Eon D-LITE+,Eon D-LITE+,Hyundai,2017,Petrol,"27,803",Manual,₹2.14 Lakh,Kolkata
104,Maruti Alto K10 VXI PLUS,Alto K10 VXI PLUS,Maruti,2022,Petrol,"2,821",Manual,₹4.71 Lakh,Kolkata
105,Hyundai NEW SANTRO MAGNA,NEW SANTRO MAGNA,Hyundai,2018,Petrol,"30,271",Manual,₹3.75 Lakh,Kolkata
106,Maruti S PRESSO LXI (O),S PRESSO LXI (O),Maruti,2019,Petrol,"36,132",Manual,₹3.70 Lakh,Kolkata
107,Maruti Wagon R 1.0 VXI,Wagon R 1.0 VXI,Maruti,2012,Petrol,"60,764",Manual,₹1.90 Lakh,Kolkata
108,Tata Tiago XT PETROL,Tiago XT PETROL,Tata,2021,Petrol,"11,624",Manual,₹5.17 Lakh,Kolkata
